## Observations
### 1. Analysis of data provided showed that charter schools had a better performance in all analyzed metrics than district schools. 
### 2. Small and medium sized schools (less than 2,000 students) had a better performance than large sized schools.
### 3. All schools but Pena High School had a better average reading score than an average math score.
### 4. Schools with budgets per student in the categories below 615 USD per student had better performances than schools with greater budgets.


In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np 
import re

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_combined_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


## District Summary

In [2]:
total_schools = len(school_combined_df['school_name'].unique().tolist())
total_students = len(school_combined_df['Student ID'].unique().tolist())
#Calculate the total budget
total_budget = school_combined_df['budget'].sum()
#Calculate the average math score and store in a variable
math_score_mean = school_combined_df['math_score'].mean()
#Calculate the average reading score and store in a variable
reading_score_mean = school_combined_df['reading_score'].mean()
#Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
rate_overall_passing = (reading_score_mean + math_score_mean)/2
#Calculate the percentage of students with a passing math score (70 or greater)
pass_math = (school_combined_df['math_score'] >= 70).sum()
#passing_math_rate = pass_math/total_math * 100
rate_passing_math = (pass_math/total_students)*100
#Calculate the percentage of students with a passing reading score (70 or greater)
pass_reading = (school_combined_df['reading_score'] >= 70).sum()
rate_passing_reading = (pass_reading/total_students)*100
#Create a dataframe to hold the above results
district_summary_df = pd.DataFrame([{'Total Schools': total_schools,
                       'Total Stuidents':total_students,
                       'Total Budget (USD Billions)': total_budget/1000000000,
                       'Average Math Score': math_score_mean,
                       'Average Reading Score': reading_score_mean,
                       'Passing Math (%)': rate_passing_math,
                       'Passing Reading (%)': rate_passing_reading,
                       'Overall Passing Rate (%)': rate_overall_passing}])

district_summary_df.round(2)

#Optional: give the displayed data cleaner formatting

,Total Schools,Total Stuidents,Total Budget (USD Billions),Average Math Score,Average Reading Score,Passing Math (%),Passing Reading (%),Overall Passing Rate (%)
0,15,39170,82.93,78.99,81.88,74.98,85.81,80.43


## School Summary Exercises

In [3]:
#get school name, size, type and budget from school_data csv
gen_info_df = school_data.copy()

#get average scores with groupby
sch_grp =  school_combined_df.groupby(['school_name'])
math_mean_sch = sch_grp.mean()['math_score']
read_mean_sch = sch_grp.mean()['reading_score']
#merge averages of math and reading
scores_mean_sch_df = pd.merge(math_mean_sch,read_mean_sch, how="left", on=["school_name"])
#filter scores, calculate pass math with grouby.count.size
pass_math_df = school_combined_df.loc[school_combined_df["math_score"] >= 70]
pass_math_sch_grp = pass_math_df.groupby(['school_name']).count()['size']
pass_math_sch = pd.DataFrame(pass_math_df.groupby(['school_name']).count()['size'])
#filter scores, calculate reading pass with grouby.count.size
pass_read_df = school_combined_df.loc[school_combined_df["reading_score"] >= 70]
pass_read_sch_grp = pass_read_df.groupby(['school_name']).count()['size']
pass_read_sch = pd.DataFrame(pass_read_df.groupby(['school_name']).count()['size'])
#merge counts of passing math and reading, and create columns
scores_count_sch_df = pd.merge(pass_math_sch, pass_read_sch, how="left", on=["school_name"])
scores_count_sch_df = scores_count_sch_df.rename(columns={"size_x": "Math Pass", "size_y": "Reading Pass"})
scores_count_sch_df
#merge school data with scores
gen_info_scores_df = pd.merge(gen_info_df, scores_mean_sch_df, how="left", on=["school_name"])
# merge with counts
gen_info_passing_df = pd.merge(gen_info_scores_df, scores_count_sch_df, how="left", on=["school_name"])
del gen_info_passing_df['School ID']
gen_info_passing_df

# add columns and calculate rates 
gen_info_passing_df['Per Student Budget (USD)'] = (gen_info_passing_df['budget']/gen_info_passing_df['size'])
gen_info_passing_df['% Passing Math'] = (gen_info_passing_df['Math Pass'] /  gen_info_passing_df['size'])*100
gen_info_passing_df['% Passing Reading'] = (gen_info_passing_df['Reading Pass'] /  gen_info_passing_df['size'])*100
gen_info_passing_df['Overall Passing Rate'] = (gen_info_passing_df['% Passing Math'] + gen_info_passing_df['% Passing Reading'])/2
gen_info_passing_df = gen_info_passing_df.rename(columns={"school_name": "School Name", \
                                                          "type": "Type", 'size':'Total Students',\
                                                         'budget': 'Total Budget (USD)',\
                                                         'math_score':'Average Math Score',\
                                                         'reading_score':'Average Reading Score',\
                                                         })
gen_info_schools = gen_info_passing_df.copy()
gen_info_schools.round(2)

,School Name,Type,Total Students,Total Budget (USD),Average Math Score,Average Reading Score,Math Pass,Reading Pass,Per Student Budget (USD),% Passing Math,% Passing Reading,Overall Passing Rate
0,Huang High School,District,2917,1910635,76.63,81.18,1916,2372,655.0,65.68,81.32,73.50
1,Figueroa High School,District,2949,1884411,76.71,81.16,1946,2381,639.0,65.99,80.74,73.36
2,Shelton High School,Charter,1761,1056600,83.36,83.73,1653,1688,600.0,93.87,95.85,94.86
3,Hernandez High School,District,4635,3022020,77.29,80.93,3094,3748,652.0,66.75,80.86,73.81
4,Griffin High School,Charter,1468,917500,83.35,83.82,1371,1426,625.0,93.39,97.14,95.27
5,Wilson High School,Charter,2283,1319574,83.27,83.99,2143,2204,578.0,93.87,96.54,95.20
6,Cabrera High School,Charter,1858,1081356,83.06,83.98,1749,1803,582.0,94.13,97.04,95.59
7,Bailey High School,District,4976,3124928,77.05,81.03,3318,4077,628.0,66.68,81.93,74.31
8,Holden High School,Charter,427,248087,83.80,83.81,395,411,581.0,92.51,96.25,94.38
9,Pena High School,Charter,962,585858,83.84,84.04,910,923,609.0,94.59,95.95,95.27


## Top Performing Schools (By Passing Rate)


In [4]:
top_five = gen_info_schools.sort_values(by=['Overall Passing Rate'], ascending=False).head(5)
top_five.round(2)

,School Name,Type,Total Students,Total Budget (USD),Average Math Score,Average Reading Score,Math Pass,Reading Pass,Per Student Budget (USD),% Passing Math,% Passing Reading,Overall Passing Rate
6,Cabrera High School,Charter,1858,1081356,83.06,83.98,1749,1803,582.0,94.13,97.04,95.59
14,Thomas High School,Charter,1635,1043130,83.42,83.85,1525,1591,638.0,93.27,97.31,95.29
9,Pena High School,Charter,962,585858,83.84,84.04,910,923,609.0,94.59,95.95,95.27
4,Griffin High School,Charter,1468,917500,83.35,83.82,1371,1426,625.0,93.39,97.14,95.27
5,Wilson High School,Charter,2283,1319574,83.27,83.99,2143,2204,578.0,93.87,96.54,95.20


## Bottom Performing Schools (By Passing Rate)

In [5]:
bottom_five = gen_info_schools.sort_values(by=['Overall Passing Rate'], ascending=True).head(5)
bottom_five.round(2)

,School Name,Type,Total Students,Total Budget (USD),Average Math Score,Average Reading Score,Math Pass,Reading Pass,Per Student Budget (USD),% Passing Math,% Passing Reading,Overall Passing Rate
11,Rodriguez High School,District,3999,2547363,76.84,80.74,2654,3208,637.0,66.37,80.22,73.29
1,Figueroa High School,District,2949,1884411,76.71,81.16,1946,2381,639.0,65.99,80.74,73.36
0,Huang High School,District,2917,1910635,76.63,81.18,1916,2372,655.0,65.68,81.32,73.50
12,Johnson High School,District,4761,3094650,77.07,80.97,3145,3867,650.0,66.06,81.22,73.64
13,Ford High School,District,2739,1763916,77.10,80.75,1871,2172,644.0,68.31,79.30,73.80


## Math Scores by Grade

In [6]:
#get grade, school and math columns from the df
school_combined_df[['grade', 'school_name', 'math_score']]

math_by_grade = school_combined_df[['grade', 'school_name', 'math_score']]

average_nine_df= math_by_grade.loc[math_by_grade["grade"].str.contains('9th')]
average_nine_grade = pd.DataFrame(average_nine_df.groupby(['school_name']).mean())
average_nine_grade = average_nine_grade.rename(columns={"math_score": "9th"})

average_ten_df = math_by_grade.loc[math_by_grade["grade"].str.contains('10th')]
average_ten_grade = pd.DataFrame(average_ten_df.groupby(['school_name']).mean())
average_ten_grade = average_ten_grade.rename(columns={"math_score": "10th"})

average_eleven_df = math_by_grade.loc[math_by_grade["grade"].str.contains('11th')]
average_eleven_grade = pd.DataFrame(average_eleven_df.groupby(['school_name']).mean())
average_eleven_grade = average_eleven_grade.rename(columns={"math_score": "11th"})

average_twelve_df = math_by_grade.loc[math_by_grade["grade"].str.contains('12th')]
average_twelve_grade = pd.DataFrame(average_twelve_df.groupby(['school_name']).mean())
average_twelve_grade = average_twelve_grade.rename(columns={"math_score": "12th"})
#merge filtered data
math_by_level = pd.merge(average_nine_grade,average_ten_grade , how="left", on=["school_name"])
math_by_level = pd.merge(math_by_level,average_eleven_grade , how="left", on=["school_name"])
math_by_level = pd.merge(math_by_level,average_twelve_grade , how="left", on=["school_name"])
#print my table
math_by_level.round(2)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

In [7]:
#get grade, school and reading columns from the df
school_combined_df[['grade', 'school_name', 'reading_score']]

reading_by_grade = school_combined_df[['grade', 'school_name', 'reading_score']]
#filter grades by school level
average_nine_read_df = reading_by_grade.loc[reading_by_grade["grade"].str.contains('9th')]
average_nine_read_grade = pd.DataFrame(average_nine_read_df.groupby(['school_name']).mean())
average_nine_read_grade = average_nine_read_grade.rename(columns={"reading_score": "9th"})

average_ten_read_df = reading_by_grade.loc[reading_by_grade["grade"].str.contains('10th')]
average_ten_read_grade = pd.DataFrame(average_ten_read_df.groupby(['school_name']).mean())
average_ten_read_grade = average_ten_read_grade.rename(columns={"reading_score": "10th"})

average_eleven_read_df= reading_by_grade.loc[reading_by_grade["grade"].str.contains('11th')]
average_eleven_read_grade = pd.DataFrame(average_eleven_read_df.groupby(['school_name']).mean())
average_eleven_read_grade = average_eleven_read_grade.rename(columns={"reading_score": "11th"})

average_twelve_read_df= reading_by_grade.loc[reading_by_grade["grade"].str.contains('12th')]
average_twelve_read_grade = pd.DataFrame(average_twelve_read_df.groupby(['school_name']).mean())
average_twelve_read_grade = average_twelve_read_grade.rename(columns={"reading_score": "12th"})
#merge filtered data
reading_by_level = pd.merge(average_nine_read_grade,average_ten_read_grade, how="left", on=["school_name"])
reading_by_level = pd.merge(reading_by_level, average_eleven_read_grade, how="left", on=["school_name"])
reading_by_level = pd.merge(reading_by_level, average_twelve_read_grade, how="left", on=["school_name"])
#print my table
reading_by_level.round(2)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending


In [8]:
# Set bins and name groups
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
#create a column and cut 
gen_info_schools["Spending Ranges(per student)"]= pd.cut(gen_info_schools["Per Student Budget (USD)"], \
                                                          spending_bins, labels=group_names, \
                                                          include_lowest=True)
spending_per_stu = gen_info_schools[['Spending Ranges(per student)','School Name','Average Math Score', 'Average Reading Score',\
                                    '% Passing Math', '% Passing Reading', 'Overall Passing Rate']]

spending_per_stu = spending_per_stu.groupby('Spending Ranges(per student)').mean()
spending_per_stu.round(2)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges(per student),,,,,
<$585,83.46,83.93,93.46,96.61,95.04
$585-615,83.60,83.89,94.23,95.90,95.07
$615-645,79.08,81.89,75.67,86.11,80.89
$645-675,77.00,81.03,66.16,81.13,73.65


## Scores by School Size


In [9]:
#Set sample bins and name groups
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
#create a column and cut 
gen_info_schools["School Size"]= pd.cut(gen_info_schools["Total Students"], \
                                                          size_bins, labels=group_names, \
                                                          include_lowest=True)


school_size = gen_info_schools[['School Size','School Name','Average Math Score', 'Average Reading Score',\
                                    '% Passing Math', '% Passing Reading', 'Overall Passing Rate']]
#groupby School Size
school_size_gpr = school_size.groupby('School Size').mean()
school_size_gpr.round(2)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,94.82
Medium (1000-2000),83.37,83.86,93.60,96.79,95.20
Large (2000-5000),77.75,81.34,69.96,82.77,76.36


## Scores by School Type

In [10]:
school_type = gen_info_schools[['Type','School Name','Average Math Score', 'Average Reading Score',\
                                    '% Passing Math', '% Passing Reading', 'Overall Passing Rate']]

#groupby type
school_type = school_type.groupby('Type').mean()
school_type.round(2)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type,,,,,
Charter,83.47,83.90,93.62,96.59,95.10
District,76.96,80.97,66.55,80.80,73.67
